In [7]:
import numpy as np
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import generate_uid, ImplicitVRLittleEndian
from datetime import datetime, date
import os
import glob
from pathlib import Path

def find_npy_files(directory):
    """
    Find .npy files in a directory
    Returns list of .npy files found
    """
    directory = Path(directory)
    npy_files = []
    
    if directory.is_dir():
        # Search for .npy files
        npy_files.extend(list(directory.glob('*.npy')))
        npy_files.extend(list(directory.glob('*.NPY')))
        
        # Search in subdirectories
        for subdir in directory.iterdir():
            if subdir.is_dir():
                try:
                    npy_files.extend(list(subdir.glob('*.npy')))
                    npy_files.extend(list(subdir.glob('*.NPY')))
                except:
                    continue
    
    return [str(f) for f in npy_files]

def find_dicom_files(directory):
    """
    Find DICOM files in a directory
    Returns the first valid DICOM file found
    """
    directory = Path(directory)
    
    # Common DICOM file extensions
    extensions = ['*.dcm', '*.DCM', '*.dicom', '*.DICOM', '*']
    
    for ext in extensions:
        files = list(directory.glob(ext))
        for file_path in files:
            try:
                # Try to read as DICOM
                ds = pydicom.dcmread(str(file_path), force=True)
                print(f"Found DICOM file: {file_path}")
                return str(file_path)
            except Exception as e:
                continue
    
    # If no files found with extensions, try all files
    if directory.is_dir():
        for file_path in directory.iterdir():
            if file_path.is_file():
                try:
                    ds = pydicom.dcmread(str(file_path), force=True)
                    print(f"Found DICOM file: {file_path}")
                    return str(file_path)
                except Exception as e:
                    continue
    
    raise FileNotFoundError(f"No valid DICOM files found in {directory}")

class NPYtoDICOMConverter:
    def __init__(self, original_dicom_path, npy_data_path):
        """
        Initialize converter with original DICOM and NPY data
        
        Args:
            original_dicom_path: Path to original DICOM file or directory containing DICOM files
            npy_data_path: Path to .npy file or directory containing .npy files
        """
        # Handle directory or file path for DICOM
        if os.path.isdir(original_dicom_path):
            print(f"Searching for DICOM files in directory: {original_dicom_path}")
            dicom_file = find_dicom_files(original_dicom_path)
            self.original_ds = pydicom.dcmread(dicom_file, force=True)
        else:
            self.original_ds = pydicom.dcmread(original_dicom_path, force=True)
        
        # Handle directory or file path for NPY
        if os.path.isdir(npy_data_path):
            print(f"Searching for .npy files in directory: {npy_data_path}")
            npy_files = find_npy_files(npy_data_path)
            if not npy_files:
                raise FileNotFoundError(f"No .npy files found in {npy_data_path}")
            
            print("Found .npy files:")
            for i, npy_file in enumerate(npy_files):
                print(f"  {i+1}. {npy_file}")
            
            # Use the first .npy file found (you can modify this logic)
            npy_file_path = npy_files[0]
            print(f"Using: {npy_file_path}")
            self.npy_data = np.load(npy_file_path)
        elif os.path.isfile(npy_data_path):
            self.npy_data = np.load(npy_data_path)
        else:
            raise FileNotFoundError(f"NPY file not found: {npy_data_path}")
        
        # Get current date/time for DICOM headers
        self.dt = datetime.now()
        self.date_str = self.dt.strftime('%Y%m%d')
        self.time_str = self.dt.strftime('%H%M%S.%f')
        
    def create_rt_dose(self, output_path='RD.dcm'):
        """
        Create RT Dose DICOM file from NPY data
        """
        # Create new dataset
        ds = Dataset()
        
        # Copy patient and study information from original
        ds.PatientName = self.original_ds.get('PatientName', 'Anonymous')
        ds.PatientID = self.original_ds.get('PatientID', 'ID001')
        ds.PatientBirthDate = self.original_ds.get('PatientBirthDate', '')
        ds.PatientSex = self.original_ds.get('PatientSex', 'O')
        
        # Study information
        ds.StudyInstanceUID = self.original_ds.get('StudyInstanceUID', generate_uid())
        ds.StudyDate = self.original_ds.get('StudyDate', self.date_str)
        ds.StudyTime = self.original_ds.get('StudyTime', self.time_str)
        ds.StudyID = self.original_ds.get('StudyID', '1')
        ds.AccessionNumber = self.original_ds.get('AccessionNumber', '')
        
        # Series information for RT Dose
        ds.SeriesInstanceUID = generate_uid()
        ds.SeriesNumber = '999'
        ds.SeriesDate = self.date_str
        ds.SeriesTime = self.time_str
        ds.SeriesDescription = 'RT Dose from Post-Processing'
        
        # Instance information
        ds.SOPInstanceUID = generate_uid()
        ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.481.2'  # RT Dose Storage
        ds.InstanceNumber = '1'
        ds.InstanceCreationDate = self.date_str
        ds.InstanceCreationTime = self.time_str
        
        # RT Dose specific attributes
        ds.Modality = 'RTDOSE'
        ds.DoseUnits = 'GY'  # Gray units
        ds.DoseType = 'PHYSICAL'
        ds.DoseSummationType = 'PLAN'
        
        # Image orientation and position (copy from original if available)
        if hasattr(self.original_ds, 'ImageOrientationPatient'):
            ds.ImageOrientationPatient = self.original_ds.ImageOrientationPatient
        else:
            ds.ImageOrientationPatient = [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
            
        if hasattr(self.original_ds, 'ImagePositionPatient'):
            ds.ImagePositionPatient = self.original_ds.ImagePositionPatient
        else:
            ds.ImagePositionPatient = [0.0, 0.0, 0.0]
        
        # Pixel spacing (copy from original or set default)
        if hasattr(self.original_ds, 'PixelSpacing'):
            ds.PixelSpacing = self.original_ds.PixelSpacing
        else:
            ds.PixelSpacing = [1.0, 1.0]
            
        # Slice thickness
        if hasattr(self.original_ds, 'SliceThickness'):
            ds.SliceThickness = self.original_ds.SliceThickness
        else:
            ds.SliceThickness = 1.0
        
        # Frame of reference
        if hasattr(self.original_ds, 'FrameOfReferenceUID'):
            ds.FrameOfReferenceUID = self.original_ds.FrameOfReferenceUID
        else:
            ds.FrameOfReferenceUID = generate_uid()
        
        # Dose grid scaling
        dose_max = np.max(self.npy_data)
        ds.DoseGridScaling = float(dose_max / 65535)  # Scale to use full 16-bit range
        
        # Image data attributes - use original DICOM dimensions
        if hasattr(self.original_ds, 'Rows') and hasattr(self.original_ds, 'Columns'):
            original_rows = self.original_ds.Rows
            original_cols = self.original_ds.Columns
            print(f"Original DICOM dimensions: {original_rows} x {original_cols}")
        else:
            original_rows = self.npy_data.shape[-2]  # Height
            original_cols = self.npy_data.shape[-1]  # Width
            print(f"Using NPY dimensions as original: {original_rows} x {original_cols}")
        
        # Use NPY data dimensions (after unpadding)
        ds.Rows = self.npy_data.shape[-2]
        ds.Columns = self.npy_data.shape[-1]
        print(f"RT Dose dimensions: {ds.Rows} x {ds.Columns}")
        
        # Handle number of frames
        if len(self.npy_data.shape) == 3:
            if hasattr(self.original_ds, 'NumberOfFrames'):
                original_frames = int(self.original_ds.NumberOfFrames)
                print(f"Original number of frames: {original_frames}")
            else:
                original_frames = self.npy_data.shape[0]
                print(f"Using NPY number of slices as original: {original_frames}")
            
            ds.NumberOfFrames = self.npy_data.shape[0]
            print(f"RT Dose number of frames: {ds.NumberOfFrames}")
        else:
            ds.NumberOfFrames = 1
        
        # ปรับ ImagePositionPatient ให้สอดคล้องกับขนาดที่เปลี่ยนไป
        if (hasattr(self.original_ds, 'ImagePositionPatient') and 
            hasattr(self.original_ds, 'PixelSpacing') and
            hasattr(self.original_ds, 'Rows') and 
            hasattr(self.original_ds, 'Columns')):
            
            current_rows = self.npy_data.shape[-2]
            current_cols = self.npy_data.shape[-1]
            
            # ถ้าขนาดต่างกัน ปรับ ImagePositionPatient
            if current_rows != original_rows or current_cols != original_cols:
                pixel_spacing = self.original_ds.PixelSpacing
                
                # คำนวณ offset (สมมติว่า crop จาก center)
                row_offset = (original_rows - current_rows) // 2 * float(pixel_spacing[0])
                col_offset = (original_cols - current_cols) // 2 * float(pixel_spacing[1])
                
                # ปรับ ImagePositionPatient
                original_pos = list(self.original_ds.ImagePositionPatient)
                ds.ImagePositionPatient = [
                    original_pos[0] + col_offset,  # X
                    original_pos[1] + row_offset,  # Y  
                    original_pos[2]                # Z (ไม่เปลี่ยน)
                ]
                
                print(f"Adjusted ImagePositionPatient for TPS alignment:")
                print(f"  Original position: {original_pos}")
                print(f"  Adjusted position: {ds.ImagePositionPatient}")
                print(f"  Offset applied: X={col_offset:.2f}mm, Y={row_offset:.2f}mm")
                print(f"  This ensures correct positioning in TPS without padding")
        
        # Grid frame offset vector for multi-frame
        if len(self.npy_data.shape) == 3:
            if hasattr(self.original_ds, 'SliceThickness'):
                slice_spacing = float(self.original_ds.SliceThickness)
            elif hasattr(self.original_ds, 'SpacingBetweenSlices'):
                slice_spacing = float(self.original_ds.SpacingBetweenSlices)
            else:
                slice_spacing = 1.0
            
            print(f"Using slice spacing: {slice_spacing} mm")
            ds.GridFrameOffsetVector = [i * slice_spacing for i in range(ds.NumberOfFrames)]
        else:
            ds.GridFrameOffsetVector = [0.0]
        
        ds.BitsAllocated = 16
        ds.BitsStored = 16
        ds.HighBit = 15
        ds.PixelRepresentation = 0
        ds.SamplesPerPixel = 1
        ds.PhotometricInterpretation = 'MONOCHROME2'
        
        # Convert and scale pixel data
        scaled_data = (self.npy_data / dose_max * 65535).astype(np.uint16)
        
        print(f"Final RT Dose shape: {scaled_data.shape}")
        print(f"Dose scaling factor: {ds.DoseGridScaling}")
        print(f"Max dose value: {dose_max}")
        
        ds.PixelData = scaled_data.tobytes()
        
        # Create file dataset
        file_meta = Dataset()
        file_meta.MediaStorageSOPClassUID = ds.SOPClassUID
        file_meta.MediaStorageSOPInstanceUID = ds.SOPInstanceUID
        file_meta.ImplementationClassUID = generate_uid()
        file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        
        file_ds = FileDataset(output_path, ds, file_meta, 
                             preamble=b"\0" * 128,  # Add 128-byte preamble
                             is_implicit_VR=True, 
                             is_little_endian=True)
        file_ds.save_as(output_path, write_like_original=False)
        
        print(f"RT Dose DICOM saved as: {output_path}")
        return output_path
    
    def create_rt_plan(self, output_path='RP.dcm', beam_name='PostPro_Beam'):
        """
        Create RT Plan DICOM file
        """
        # Create new dataset
        ds = Dataset()
        
        # Copy patient and study information from original
        ds.PatientName = self.original_ds.get('PatientName', 'Anonymous')
        ds.PatientID = self.original_ds.get('PatientID', 'ID001')
        ds.PatientBirthDate = self.original_ds.get('PatientBirthDate', '')
        ds.PatientSex = self.original_ds.get('PatientSex', 'O')
        
        # Study information
        ds.StudyInstanceUID = self.original_ds.get('StudyInstanceUID', generate_uid())
        ds.StudyDate = self.original_ds.get('StudyDate', self.date_str)
        ds.StudyTime = self.original_ds.get('StudyTime', self.time_str)
        ds.StudyID = self.original_ds.get('StudyID', '1')
        ds.AccessionNumber = self.original_ds.get('AccessionNumber', '')
        
        # Series information for RT Plan
        ds.SeriesInstanceUID = generate_uid()
        ds.SeriesNumber = '998'
        ds.SeriesDate = self.date_str
        ds.SeriesTime = self.time_str
        ds.SeriesDescription = 'RT Plan from Post-Processing'
        
        # Instance information
        ds.SOPInstanceUID = generate_uid()
        ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.481.5'  # RT Plan Storage
        ds.InstanceNumber = '1'
        ds.InstanceCreationDate = self.date_str
        ds.InstanceCreationTime = self.time_str
        
        # RT Plan specific attributes
        ds.Modality = 'RTPLAN'
        ds.RTPlanLabel = 'PostProcessing_Plan'
        ds.RTPlanName = 'Post-Processing Treatment Plan'
        ds.RTPlanDate = self.date_str
        ds.RTPlanTime = self.time_str
        ds.TreatmentProtocols = ['Post-Processing Protocol']
        ds.PlanIntent = 'RESEARCH'
        ds.RTPlanGeometry = 'PATIENT'
        
        # Frame of reference
        if hasattr(self.original_ds, 'FrameOfReferenceUID'):
            ds.FrameOfReferenceUID = self.original_ds.FrameOfReferenceUID
        else:
            ds.FrameOfReferenceUID = generate_uid()
        
        # Patient setup sequence (basic)
        patient_setup = Dataset()
        patient_setup.PatientPosition = self.original_ds.get('PatientPosition', 'HFS')
        patient_setup.PatientSetupNumber = '1'
        patient_setup.SetupTechnique = 'ISOCENTRIC'
        ds.PatientSetupSequence = [patient_setup]
        
        # Beam sequence (simplified)
        beam = Dataset()
        beam.BeamNumber = '1'
        beam.BeamName = beam_name
        beam.BeamDescription = 'Post-processed beam'
        beam.BeamType = 'STATIC'
        beam.RadiationType = 'PHOTON'
        beam.TreatmentMachineType = 'RESEARCH'
        beam.PrimaryDosimeterUnit = 'MU'
        beam.SourceAxisDistance = 1000.0  # mm
        beam.NumberOfWedges = '0'
        beam.NumberOfCompensators = '0'
        beam.NumberOfBoli = '0'
        beam.NumberOfBlocks = '0'
        beam.FinalCumulativeMetersetWeight = 1.0
        beam.ReferencedPatientSetupNumber = '1'
        
        # Control point sequence (basic single control point)
        control_point = Dataset()
        control_point.ControlPointIndex = '0'
        control_point.CumulativeMetersetWeight = 0.0
        control_point.GantryAngle = 0.0
        control_point.GantryAngleDirection = 'NONE'
        control_point.BeamLimitingDeviceAngle = 0.0
        control_point.BeamLimitingDeviceAngleDirection = 'NONE'
        control_point.PatientSupportAngle = 0.0
        control_point.PatientSupportAngleDirection = 'NONE'
        control_point.TableTopEccentricAngle = 0.0
        control_point.TableTopEccentricAngleDirection = 'NONE'
        control_point.TableTopVerticalPosition = 0.0
        control_point.TableTopLongitudinalPosition = 0.0
        control_point.TableTopLateralPosition = 0.0
        control_point.IsocenterPosition = [0.0, 0.0, 0.0]
        
        beam.ControlPointSequence = [control_point]
        ds.BeamSequence = [beam]
        
        # Fraction group sequence
        fraction_group = Dataset()
        fraction_group.FractionGroupNumber = '1'
        fraction_group.NumberOfFractionsPlanned = '1'
        fraction_group.NumberOfBeams = '1'
        
        # Referenced beam sequence
        ref_beam = Dataset()
        ref_beam.ReferencedBeamNumber = '1'
        ref_beam.BeamMeterset = 100.0
        fraction_group.ReferencedBeamSequence = [ref_beam]
        
        ds.FractionGroupSequence = [fraction_group]
        
        # Create file dataset
        file_meta = Dataset()
        file_meta.MediaStorageSOPClassUID = ds.SOPClassUID
        file_meta.MediaStorageSOPInstanceUID = ds.SOPInstanceUID
        file_meta.ImplementationClassUID = generate_uid()
        file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
        
        file_ds = FileDataset(output_path, ds, file_meta, 
                             preamble=b"\0" * 128,  # Add 128-byte preamble
                             is_implicit_VR=True, 
                             is_little_endian=True)
        file_ds.save_as(output_path, write_like_original=False)
        
        print(f"RT Plan DICOM saved as: {output_path}")
        return output_path

def align_with_original_ct(rd_path, rp_path, original_ct_path):
    """
    Function to align RT Dose and RT Plan with original CT
    This ensures proper geometric alignment in TPS
    """
    # Handle directory or file path for CT
    if os.path.isdir(original_ct_path):
        print(f"Searching for CT DICOM files in directory: {original_ct_path}")
        ct_file = find_dicom_files(original_ct_path)
        ct_ds = pydicom.dcmread(ct_file, force=True)
    else:
        ct_ds = pydicom.dcmread(original_ct_path, force=True)
    
    # Read RT Dose and RT Plan
    rd_ds = pydicom.dcmread(rd_path)
    rp_ds = pydicom.dcmread(rp_path)
    
    # Copy geometric information from CT to RT structures
    geometric_tags = [
        'ImageOrientationPatient',
        'PixelSpacing',
        'SliceThickness',
        'FrameOfReferenceUID'
    ]
    
    for tag in geometric_tags:
        if hasattr(ct_ds, tag):
            ct_value = getattr(ct_ds, tag)
            
            # Update RT Dose (except ImagePositionPatient which was already adjusted)
            if hasattr(rd_ds, tag) and tag != 'ImagePositionPatient':
                setattr(rd_ds, tag, ct_value)
            
            # Update RT Plan frame of reference
            if tag == 'FrameOfReferenceUID' and hasattr(rp_ds, tag):
                setattr(rp_ds, tag, ct_value)
    
    # Keep the adjusted ImagePositionPatient from RT Dose creation
    print("Keeping adjusted ImagePositionPatient for proper TPS positioning")
    
    # Save aligned files
    rd_aligned_path = rd_path.replace('.dcm', '_aligned.dcm')
    rp_aligned_path = rp_path.replace('.dcm', '_aligned.dcm')
    
    rd_ds.save_as(rd_aligned_path)
    rp_ds.save_as(rp_aligned_path)
    
    print(f"Aligned RT Dose saved as: {rd_aligned_path}")
    print(f"Aligned RT Plan saved as: {rp_aligned_path}")
    
    return rd_aligned_path, rp_aligned_path

# Usage example
if __name__ == "__main__":
    # Example usage - can handle both files and directories
    original_dicom_path = r"d:\Workhard\OneDrive_1_30-5-2568\GT\BT033"  # Directory or file
    
    # Let's first check what's in your directory to find .npy files
    base_directory = r"d:\Workhard\OneDrive_1_30-5-2568"
    print(f"Searching for .npy files in: {base_directory}")
    
    # Search for .npy files in the base directory and subdirectories
    npy_files = find_npy_files(base_directory)
    if npy_files:
        print("Found .npy files:")
        for i, npy_file in enumerate(npy_files):
            print(f"  {i+1}. {npy_file}")
        npy_data_path = npy_files[0]  # Use the first one found
        print(f"Using .npy file: {npy_data_path}")
    else:
        print("No .npy files found. Please specify the correct path.")
        print("Looking for files in current directory...")
        current_dir_npy = find_npy_files(".")
        if current_dir_npy:
            print("Found .npy files in current directory:")
            for npy_file in current_dir_npy:
                print(f"  - {npy_file}")
        else:
            print("No .npy files found in current directory either.")
        npy_data_path = input("Please enter the full path to your .npy file: ")
    
    original_ct_path = r"d:\Workhard\OneDrive_1_30-5-2568\GT\BT033_CT"  # Directory or file
    
    try:
        # Initialize converter
        print("\nInitializing converter...")
        converter = NPYtoDICOMConverter(original_dicom_path, npy_data_path)
        
        print(f"NPY data shape: {converter.npy_data.shape}")
        print(f"NPY data type: {converter.npy_data.dtype}")
        print(f"NPY data range: {np.min(converter.npy_data)} to {np.max(converter.npy_data)}")
        
        # Print original DICOM info
        if hasattr(converter.original_ds, 'Rows'):
            print(f"Original DICOM dimensions: {converter.original_ds.Rows} x {converter.original_ds.Columns}")
        if hasattr(converter.original_ds, 'NumberOfFrames'):
            print(f"Original DICOM frames: {converter.original_ds.NumberOfFrames}")
        elif hasattr(converter.original_ds, 'SliceThickness'):
            print(f"Original DICOM slice thickness: {converter.original_ds.SliceThickness}")
        
        print(f"Patient Position: {getattr(converter.original_ds, 'PatientPosition', 'Not specified')}")
        print(f"Modality: {getattr(converter.original_ds, 'Modality', 'Not specified')}")
        
        # Create RT Dose and RT Plan files
        print("\nCreating RT Dose...")
        rd_path = converter.create_rt_dose('RD.dcm')
        
        print("Creating RT Plan...")
        rp_path = converter.create_rt_plan('RP.dcm')
        
        # Align with original CT
        print("Aligning with original CT...")
        rd_aligned, rp_aligned = align_with_original_ct(rd_path, rp_path, original_ct_path)
        
        print("\nConversion completed successfully!")
        print(f"Files ready for TPS import:")
        print(f"- RT Dose (aligned): {rd_aligned}")
        print(f"- RT Plan (aligned): {rp_aligned}")
        print("\nThe RT Dose will display at correct position in TPS without needing padding!")
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        print("\nTroubleshooting:")
        print("1. Make sure your .npy file exists")
        print("2. Check DICOM directory paths")
        print("3. Verify file permissions")
        
        # Show directory contents for debugging
        try:
            print(f"\nContents of {base_directory}:")
            for item in os.listdir(base_directory):
                item_path = os.path.join(base_directory, item)
                if os.path.isfile(item_path):
                    print(f"  FILE: {item}")
                elif os.path.isdir(item_path):
                    print(f"  DIR:  {item}")
        except Exception as e2:
            print(f"Cannot list directory contents: {e2}")

Searching for .npy files in: d:\Workhard\OneDrive_1_30-5-2568
Found .npy files:
  1. d:\Workhard\OneDrive_1_30-5-2568\BT033_postprocessed.npy
  2. d:\Workhard\OneDrive_1_30-5-2568\BT033_postprocessed.npy
Using .npy file: d:\Workhard\OneDrive_1_30-5-2568\BT033_postprocessed.npy

Initializing converter...
Searching for DICOM files in directory: d:\Workhard\OneDrive_1_30-5-2568\GT\BT033
Found DICOM file: d:\Workhard\OneDrive_1_30-5-2568\GT\BT033\CT.1.2.246.352.221.4639643498212235395.10429152843694438323.dcm
NPY data shape: (140, 133, 133)
NPY data type: float32
NPY data range: 12688.435546875 to 51815336.0
Original DICOM dimensions: 512 x 512
Original DICOM slice thickness: 2
Patient Position: HFS
Modality: CT

Creating RT Dose...
Original DICOM dimensions: 512 x 512
RT Dose dimensions: 133 x 133
Using NPY number of slices as original: 140
RT Dose number of frames: 140
Adjusted ImagePositionPatient for TPS alignment:
  Original position: ['-167.0', '-84.0', '-1326.5']
  Adjusted position

In [ ]:
# สามารถใช้ได้ทั้ง directory และ file path
original_dicom_path = r"d:\Workhard\OneDrive_1_30-5-2568\GT\BT033"
npy_data_path = r"path\to\your\postprocessed_data.npy"
original_ct_path = r"d:\Workhard\OneDrive_1_30-5-2568\GT\BT033_CT"

# รันโค้ด
converter = NPYtoDICOMConverter(original_dicom_path, npy_data_path)

PermissionError: [Errno 13] Permission denied: 'd:\\Workhard\\OneDrive_1_30-5-2568\\GT\\BT033'